# Day 3: Position Sizing

## Learning Objectives
- Implement Kelly Criterion
- Volatility-based position sizing
- Risk budgeting approaches

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("✅ Libraries loaded!")
print("📚 Day 3: Position Sizing")

In [ ]:
# ============================================================
# PREPARE DATA
# ============================================================

print("PREPARING DATA")
print("="*60)

np.random.seed(42)
n_days = 1500

returns = np.random.normal(0.0003, 0.015, n_days)
for i in range(1, len(returns)):
    returns[i] += 0.05 * returns[i-1]

prices = 100 * np.cumprod(1 + returns)
df = pd.DataFrame({'price': prices, 'returns': returns})

df['ret_5d'] = df['price'].pct_change(5)
df['vol_5d'] = df['returns'].rolling(5).std()
df['mom_5d'] = df['returns'].rolling(5).sum()
df['target'] = (df['returns'].shift(-1) > 0).astype(int)
df = df.dropna()

feature_cols = ['ret_5d', 'vol_5d', 'mom_5d']
X = df[feature_cols].values
y = df['target'].values

split = int(len(X) * 0.7)
scaler = StandardScaler()
X_train = scaler.fit_transform(X[:split])
X_test = scaler.transform(X[split:])
y_train, y_test = y[:split], y[split:]

model = GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)[:, 1]

test_returns = returns[split + 5:split + 5 + len(predictions)]
min_len = min(len(test_returns), len(predictions))
test_returns = test_returns[:min_len]
predictions = predictions[:min_len]
probabilities = probabilities[:min_len]

print(f"Test accuracy: {(predictions == y_test[:min_len]).mean():.3f}")

## Part 1: Kelly Criterion

In [ ]:
# ============================================================
# KELLY CRITERION
# ============================================================

print("KELLY CRITERION")
print("="*60)

def kelly_fraction(win_prob, win_return, loss_return):
    """
    Calculate Kelly fraction.
    
    f* = (p * b - q) / b
    
    where:
        p = win probability
        q = 1 - p
        b = win/loss ratio (odds)
    """
    if loss_return == 0:
        return 0
    
    b = abs(win_return / loss_return)
    q = 1 - win_prob
    
    kelly = (win_prob * b - q) / b
    
    return max(0, kelly)  # Don't go negative (no shorts in basic Kelly)

# Estimate from training data
train_predictions = model.predict(X_train)
train_returns = returns[5:split + 5]

# When prediction is UP (1)
up_mask = train_predictions == 1
up_actual = train_returns[up_mask]
up_win_prob = (up_actual > 0).mean()
up_win_return = up_actual[up_actual > 0].mean() if (up_actual > 0).any() else 0
up_loss_return = up_actual[up_actual <= 0].mean() if (up_actual <= 0).any() else 0

kelly_up = kelly_fraction(up_win_prob, up_win_return, up_loss_return)

print(f"When predicting UP:")
print(f"  Win probability: {up_win_prob:.2%}")
print(f"  Avg win return: {up_win_return:.4%}")
print(f"  Avg loss return: {up_loss_return:.4%}")
print(f"  Kelly fraction: {kelly_up:.2%}")
print(f"  Half-Kelly: {kelly_up/2:.2%}")

In [ ]:
# ============================================================
# PROBABILITY-WEIGHTED KELLY
# ============================================================

print("\nPROBABILITY-WEIGHTED POSITION SIZING")
print("="*60)

def probability_kelly_size(prob, base_kelly=1.0, max_position=1.0):
    """
    Scale position by prediction confidence.
    
    Position = base_kelly * (2 * prob - 1)  # Maps [0,1] to [-1,1]
    """
    # Edge from 0.5
    edge = 2 * prob - 1
    position = base_kelly * edge
    
    # Clip to max
    position = np.clip(position, -max_position, max_position)
    
    return position

# Apply to test predictions
half_kelly = kelly_up / 2
positions_kelly = np.array([probability_kelly_size(p, base_kelly=half_kelly) for p in probabilities])

print(f"Position statistics:")
print(f"  Mean: {positions_kelly.mean():.4f}")
print(f"  Std: {positions_kelly.std():.4f}")
print(f"  Min: {positions_kelly.min():.4f}")
print(f"  Max: {positions_kelly.max():.4f}")

## Part 2: Volatility Targeting

In [ ]:
# ============================================================
# VOLATILITY TARGETING
# ============================================================

print("\nVOLATILITY TARGETING")
print("="*60)

def volatility_target_position(returns_history, target_vol=0.15, lookback=20):
    """
    Scale position to target a specific volatility.
    
    Position = target_vol / realized_vol
    """
    # Realized volatility (annualized)
    realized_vol = returns_history[-lookback:].std() * np.sqrt(252)
    
    if realized_vol > 0:
        position_scale = target_vol / realized_vol
    else:
        position_scale = 1.0
    
    # Cap leverage
    position_scale = min(position_scale, 2.0)
    
    return position_scale, realized_vol

# Apply volatility targeting
target_vol = 0.15  # 15% annualized target
vol_scales = []
realized_vols = []

for i in range(len(test_returns)):
    # Use rolling history
    history_start = max(0, split + i - 60)
    history = returns[history_start:split + i + 1]
    
    scale, real_vol = volatility_target_position(history, target_vol=target_vol)
    vol_scales.append(scale)
    realized_vols.append(real_vol)

vol_scales = np.array(vol_scales)

print(f"Target volatility: {target_vol:.0%}")
print(f"Average position scale: {vol_scales.mean():.2f}")
print(f"Average realized vol: {np.mean(realized_vols):.2%}")

## Part 3: Compare Sizing Methods

In [ ]:
# ============================================================
# COMPARE POSITION SIZING METHODS
# ============================================================

print("\nCOMPARING POSITION SIZING METHODS")
print("="*60)

# Binary positions (baseline)
positions_binary = 2 * predictions - 1

# Kelly-weighted
positions_kelly = np.array([probability_kelly_size(p, base_kelly=0.5) for p in probabilities])

# Volatility-targeted
positions_vol = positions_binary * vol_scales

# Combined
positions_combined = positions_kelly * vol_scales

# Calculate returns
returns_binary = test_returns * positions_binary
returns_kelly = test_returns * positions_kelly
returns_vol = test_returns * positions_vol
returns_combined = test_returns * positions_combined

# Performance
def calc_metrics(rets, name):
    cum = np.cumprod(1 + rets)[-1]
    sharpe = np.sqrt(252) * rets.mean() / rets.std() if rets.std() > 0 else 0
    vol = rets.std() * np.sqrt(252)
    max_dd = (np.maximum.accumulate(np.cumprod(1+rets)) - np.cumprod(1+rets)).max()
    
    print(f"{name:<15} Return: {(cum-1)*100:>6.1f}%  Sharpe: {sharpe:>5.2f}  Vol: {vol*100:>5.1f}%  MaxDD: {max_dd*100:>5.1f}%")
    return cum, sharpe, vol, max_dd

calc_metrics(returns_binary, 'Binary')
calc_metrics(returns_kelly, 'Kelly')
calc_metrics(returns_vol, 'Vol-Target')
calc_metrics(returns_combined, 'Combined')

In [ ]:
# ============================================================
# VISUALIZATION
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Cumulative returns
axes[0, 0].plot(np.cumprod(1 + returns_binary), label='Binary', linewidth=1.5)
axes[0, 0].plot(np.cumprod(1 + returns_kelly), label='Kelly', linewidth=1.5)
axes[0, 0].plot(np.cumprod(1 + returns_vol), label='Vol-Target', linewidth=1.5)
axes[0, 0].plot(np.cumprod(1 + returns_combined), label='Combined', linewidth=1.5)
axes[0, 0].set_xlabel('Days')
axes[0, 0].set_ylabel('Cumulative Return')
axes[0, 0].set_title('Position Sizing Comparison', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Position distributions
axes[0, 1].hist(positions_binary, bins=30, alpha=0.5, label='Binary')
axes[0, 1].hist(positions_kelly, bins=30, alpha=0.5, label='Kelly')
axes[0, 1].hist(positions_combined, bins=30, alpha=0.5, label='Combined')
axes[0, 1].set_xlabel('Position Size')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Position Size Distribution', fontweight='bold')
axes[0, 1].legend()

# Position over time
axes[1, 0].plot(positions_kelly, label='Kelly', alpha=0.7)
axes[1, 0].plot(vol_scales, label='Vol Scale', alpha=0.7)
axes[1, 0].set_xlabel('Days')
axes[1, 0].set_ylabel('Position/Scale')
axes[1, 0].set_title('Position Sizing Over Time', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Rolling Sharpe
window = 60
rolling_sharpe_binary = pd.Series(returns_binary).rolling(window).apply(lambda x: np.sqrt(252) * x.mean() / x.std())
rolling_sharpe_combined = pd.Series(returns_combined).rolling(window).apply(lambda x: np.sqrt(252) * x.mean() / x.std())
axes[1, 1].plot(rolling_sharpe_binary, label='Binary', alpha=0.7)
axes[1, 1].plot(rolling_sharpe_combined, label='Combined', alpha=0.7)
axes[1, 1].axhline(0, color='gray', linestyle='--')
axes[1, 1].set_xlabel('Days')
axes[1, 1].set_ylabel('Rolling Sharpe (60d)')
axes[1, 1].set_title('Rolling Sharpe Ratio', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║              DAY 3 COMPLETE: POSITION SIZING                     ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ Kelly Criterion (optimal bet size)                           ║
║  ✓ Probability-weighted sizing                                  ║
║  ✓ Volatility targeting                                         ║
║  ✓ Combined sizing approach                                     ║
╚══════════════════════════════════════════════════════════════════╝

Tomorrow: Day 4 - Risk Management
""")